### Financial Text Sentiment Analysis


In [1]:
import os
from dotenv import load_dotenv
from newsapi import NewsApiClient
from transformers import pipeline
from datetime import datetime

load_dotenv()
key = os.getenv("NEWSAPI_KEY")

newsapi = NewsApiClient(api_key=key)
pipe = pipeline("text-classification", model="ProsusAI/finbert")


today = datetime.today().strftime('%Y-%m-%d')

keyword = 'palantir'
news = newsapi.get_everything(q=keyword,
                              from_param='2025-06-02',
                              to=today,
                              language='en',
                              sort_by='relevancy',
                                )

articles = news['articles']
articles = [article for article in articles if keyword.lower() in article['title'].lower() or keyword.lower() in article['description'].lower()]
print(articles)

total_weight = 0 
weighted_sum = 0
label_map = {"positive":1, "neutral":0, "negative":-1}

for i, article in enumerate(articles):
    
    print(f"Title: {article['title']}")
    print(f"Link: {article['url']}")
    print(f"Description: {article['description']}")
    
    sentiment = pipe(article['content'])[0]
    score = label_map.get(sentiment['label'].lower(), 0)
    weight = sentiment['score']
    
    print(f"Sentiment {sentiment['label']}, Score: {sentiment['score']}")
    print('-' * 40) 
    
    weighted_sum += score * weight
    total_weight += weight


final_score = weighted_sum / total_weight if total_weight > 0 else 0
print(f"Overall Sentiment: {'Positive' if final_score > 0.2 else 'Negative' if final_score < -0.2 else 'Neutral'} {final_score}")
    



c:\Users\yusuf\OneDrive\Documents\GitHub\Predicting-Stock-Prices-Using-LSTM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


[{'source': {'id': None, 'name': 'Gizmodo.com'}, 'author': 'Lucas Ropek', 'title': 'Is MAGA Turning on Trump’s Buddy Palantir?', 'description': "Rightwing America doesn't seem too pleased that the defense contractor is playing a pivotal role in the Trump government.", 'url': 'https://gizmodo.com/is-maga-turning-on-trumps-buddy-palantir-2000610499', 'urlToImage': 'https://gizmodo.com/app/uploads/2025/01/ThielFace.jpg', 'publishedAt': '2025-06-03T20:30:29Z', 'content': 'Creepy defense contractor Palantir has a new mission: convincing Americans that it’s not actually creepy.\r\nThe company was been put on the defensive this week after the New York Times published a sto… [+4928 chars]'}, {'source': {'id': 'business-insider', 'name': 'Business Insider'}, 'author': 'Samantha Stokes', 'title': 'The Palantir job that grows startup founders', 'description': 'Forward Deployed Software Engineers at Palantir gain startup-ready skills through direct client work. Many have left the firm to become fo